In [1]:
import pysftp
import sys
import os
python_path='/home/brian/Dropbox/Python'
sys.path.append(python_path)
from gittislab import dataloc

#Code that goes from CSV filename to local path:
local_vid='/home/brian/Dropbox/Gittis Lab Data/OptoBehavior'
example_fns=['GPe_Naive_PV_ChR2_Bilateral_zone-1_AG5304-1-UZ030520DLC_resnet50_psc_analyzeMay1shuffle1_1000000.csv',
             'Str_Naive_D1_Arch_Bilateral_10x30-3mW_AG4924-1-BI102319DLC_resnet50_psc_analyzeMay1shuffle1_1000000.csv']
max_depth=6
iteration =10
pn,fn = dataloc.fn_to_path(local_vid,example_fns,max_depth,iteration)
for n,f in enumerate(fn):
    print('%s/%s' % (pn[n],f))

/home/brian/Dropbox/Gittis Lab Data/OptoBehavior/GPe/Naive/PV/ChR2/Bilateral/zone_1/AG5304_1_UZ030520/iteration_10_resnet50_psc_analyzeMay1shuffle1_1000000.csv
/home/brian/Dropbox/Gittis Lab Data/OptoBehavior/Str/Naive/D1/Arch/Bilateral/10x30_3mW/AG4924_1_BI102319/iteration_10_resnet50_psc_analyzeMay1shuffle1_1000000.csv


In [5]:
example_fns[-1]

'Str_Naive_D1_Arch_Bilateral_10x30-3mW_AG4924-1-BI102319DLC_resnet50_psc_analyzeMay1shuffle1_1000000.csv'

In [1]:
# Count a filetype on PSC:
import pysftp
import sys
import os
python_path='/home/brian/Dropbox/Python'
sys.path.append(python_path)
from gittislab import dataloc

#Initialize connection to PSC pylon5:
remote_loc='/pylon5/ib5phrp/bisett/projects/psc_analyze-BRI-2020-05-01/videos'
srvr= pysftp.Connection('data.bridges.psc.edu',username='bisett',password='deepL4bCutta',port=22)
srvr.cwd(remote_loc)
all_files=srvr.listdir()
c=0
filetype='.h5'
for f in all_files:
    if filetype in f:
        c+=1
print('There are %d %s files in %s.' % (c,filetype,remote_loc))
srvr.close()

There are 844 .h5 files in /pylon5/ib5phrp/bisett/projects/psc_analyze-BRI-2020-05-01/videos.


In [1]:
# Find .h5 (or .csv) files on the PSC server pylon5 and bring them back to correct local folder structure (if possible)
import pysftp
import sys
import os
python_path='/home/brian/Dropbox/Python'
sys.path.append(python_path)
from gittislab import dataloc

#Initialize connection to PSC pylon5:
#remote_loc='/pylon5/ib5phrp/bisett/projects/psc_analyze-BRI-2020-05-01/videos'
remote_loc='//pylon5/ib5phrp/bisett/videos'
srvr= pysftp.Connection('data.bridges.psc.edu',username='bisett',password='deepL4bCutta',port=22)
srvr.cwd(remote_loc)
all_files=srvr.listdir()
iteration = 10
local_vid='/home/brian/Dropbox/Gittis Lab Data/OptoBehavior'
max_depth=6
missed=[]
ft='.h5' #FILETYPE
copy_missed=True
new_moved=0
prev_moved=0
missed_moved=0
for i,f in enumerate(all_files):
    #If file is of the correct filetype, proceed to copy it to local directory:
    if ft in f:
        pn,fn = dataloc.fn_to_path(local_vid,[f],max_depth,iteration)
        if os.path.exists(pn[0]):
            shorter_fn='iteration_%d_dlc_analyze%s' % (iteration,ft)
            #If file not already found locally, copy it:
            if os.path.exists(pn[0]+'/'+ shorter_fn) == False:
                print('Copy %s \n\t to %s \n\t as %s.' % (f,pn[0],shorter_fn))
                srvr.get(remote_loc + '/' + f, pn[0] + '/' + shorter_fn)
                print('\tCopied!\n')
                new_moved += 1
            else: #Skip file because it has already been copied over:
                print('%s already has a %s file of that name ... skipping\n' % (pn[0],ft))
                prev_moved +=1
        else: #This file had trouble reconstructing correct directory            
            missed.append(f)
            if copy_missed == True:                
                missed_pn=local_vid + '/PSC_missing' + '/'
                missed_pnfn=missed_pn + f
                if os.path.exists(missed_pnfn) == False:
                    print('Missed file-> unclear where to put locally: \n\t%s\n\tTemporary location: %s\n' 
                          % (f,missed_pn))
                    srvr.get(remote_loc + '/' + f, missed_pnfn)
                    missed_moved += 1
                else:
                    print('Missed file %s already found in \n\t%s ... skipping\n' % (f,missed_pn))
                    missed_moved +=1
            else:
                print('Missing local directory,skipping: \n\t%s\n\t %s\n' % (f,pn[0]))

srvr.close()
missed_pn=local_vid + '/PSC_missing' + '/'
print('%d files moved or found locally in temp location %s' % ( missed_moved,missed_pn))
print('%d files already moved and skipped. %d new files moved successfully.' % (prev_moved,new_moved))
print('%d total .h5 found locally.' % ( prev_moved + new_moved + missed_moved))
print('Finished')

Copy Str_Naive_D1_Arch_Bilateral_10x10-30mW_AG6164-3-BI082420DLC_resnet50_psc_analyzeMay1shuffle1_1000000.h5 
	 to /home/brian/Dropbox/Gittis Lab Data/OptoBehavior/Str/Naive/D1/Arch/Bilateral/10x10_30mW/AG6164_3_BI082420 
	 as iteration_10_dlc_analyze.h5.
	Copied!

Copy Str_Naive_D1_Arch_Bilateral_10x10-30mW_AG6167-1-BI082420DLC_resnet50_psc_analyzeMay1shuffle1_1000000.h5 
	 to /home/brian/Dropbox/Gittis Lab Data/OptoBehavior/Str/Naive/D1/Arch/Bilateral/10x10_30mW/AG6167_1_BI082420 
	 as iteration_10_dlc_analyze.h5.
	Copied!

Copy Str_Naive_D1_Arch_Bilateral_10x10-30mW_AG6167-3-BI082420DLC_resnet50_psc_analyzeMay1shuffle1_1000000.h5 
	 to /home/brian/Dropbox/Gittis Lab Data/OptoBehavior/Str/Naive/D1/Arch/Bilateral/10x10_30mW/AG6167_3_BI082420 
	 as iteration_10_dlc_analyze.h5.
	Copied!

Copy Str_Naive_D1_Arch_Bilateral_10x10-30mW_AG6167-5-BI082420DLC_resnet50_psc_analyzeMay1shuffle1_1000000.h5 
	 to /home/brian/Dropbox/Gittis Lab Data/OptoBehavior/Str/Naive/D1/Arch/Bilateral/10x10_30mW

In [2]:
# Find slurm_ .out files and copy:

import pysftp
import sys
import os
python_path='/home/brian/Dropbox/Python'
sys.path.append(python_path)
from gittislab import dataloc

#Initialize connection to PSC pylon5:
remote_loc='/home/bisett'
srvr= pysftp.Connection('data.bridges.psc.edu',username='bisett',password='deepL4bCutta',port=22)
remote_vid='/pylon5/ib5phrp/bisett/projects/psc_analyze-BRI-2020-05-01/videos'

#First generate list of videos:
vids=[v for v in srvr.listdir(remote_vid) if '.mpg' in v]

#Then gather files in home directory
all_files=srvr.listdir(remote_loc)
iteration = 10
local_dir='/home/brian/Dropbox/PSC/Error_logs'

max_depth=6
missed=[]
ft='.out' #FILETYPE
new_moved =0
prev_moved = 0
for i,f in enumerate(all_files):
    #If file is of the correct filetype, proceed to copy it to local directory:
    if ft in f and '.swp' not in f:
        if os.path.exists(local_dir):
            #If file not already found locally, copy it:
            if os.path.exists(local_dir +'/'+ f) == False:
                print('Copy %s \n\t to %s \n\t as %s.' % (f,local_dir,f))
                srvr.get(remote_loc + '/' + f, local_dir + '/' + f)
                print('\tCopied!\n')
                new_moved += 1
            else: #Skip file because it has already been copied over:
                print('%s already has a %s file of that name ... skipping\n' % (local_dir,ft))
                prev_moved +=1
        else: #This file had trouble reconstructing correct directory            
            print("%s local directory doesn't exist" % local_dir)
srvr.close()
print('%d files already moved and skipped. %d new files moved successfully.' % (prev_moved,new_moved))
print('%d total .h5 found locally.' % ( prev_moved + new_moved ))
print('Finished')

Copy .slurm-9127108_5.out.swp 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as .slurm-9127108_5.out.swp.
	Copied!

Copy slurm-9249639_101.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_101.out.
	Copied!

Copy slurm-9249639_102.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_102.out.
	Copied!

Copy slurm-9249639_103.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_103.out.
	Copied!

Copy slurm-9249639_104.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_104.out.
	Copied!

Copy slurm-9249639_105.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_105.out.
	Copied!

Copy slurm-9249639_106.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_106.out.
	Copied!

Copy slurm-9249639_107.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_107.out.
	Copied!

Copy slurm-9249639_108.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_108.out.
	Copied!

Copy slurm-9249639_109.out 
	 t

	Copied!

Copy slurm-9249639_177.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_177.out.
	Copied!

Copy slurm-9249639_178.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_178.out.
	Copied!

Copy slurm-9249639_179.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_179.out.
	Copied!

Copy slurm-9249639_180.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_180.out.
	Copied!

Copy slurm-9249639_181.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_181.out.
	Copied!

Copy slurm-9249639_182.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_182.out.
	Copied!

Copy slurm-9249639_183.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_183.out.
	Copied!

Copy slurm-9249639_184.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_184.out.
	Copied!

Copy slurm-9249639_185.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_185.out.
	Copied!

Copy slurm-9249639_186.out 

	Copied!

Copy slurm-9249639_254.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_254.out.
	Copied!

Copy slurm-9249639_255.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_255.out.
	Copied!

Copy slurm-9249639_256.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_256.out.
	Copied!

Copy slurm-9249639_257.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_257.out.
	Copied!

Copy slurm-9249639_258.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_258.out.
	Copied!

Copy slurm-9249639_259.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_259.out.
	Copied!

Copy slurm-9249639_260.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_260.out.
	Copied!

Copy slurm-9249639_261.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_261.out.
	Copied!

Copy slurm-9249639_262.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_262.out.
	Copied!

Copy slurm-9249639_263.out 

	Copied!

Copy slurm-9249639_331.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_331.out.
	Copied!

Copy slurm-9249639_332.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_332.out.
	Copied!

Copy slurm-9249639_333.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_333.out.
	Copied!

Copy slurm-9249639_334.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_334.out.
	Copied!

Copy slurm-9249639_335.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_335.out.
	Copied!

Copy slurm-9249639_336.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_336.out.
	Copied!

Copy slurm-9249639_337.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_337.out.
	Copied!

Copy slurm-9249639_338.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_338.out.
	Copied!

Copy slurm-9249639_339.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_339.out.
	Copied!

Copy slurm-9249639_340.out 


Copy slurm-9249639_408.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_408.out.
	Copied!

Copy slurm-9249639_409.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_409.out.
	Copied!

Copy slurm-9249639_410.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_410.out.
	Copied!

Copy slurm-9249639_411.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_411.out.
	Copied!

Copy slurm-9249639_412.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_412.out.
	Copied!

Copy slurm-9249639_413.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_413.out.
	Copied!

Copy slurm-9249639_414.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_414.out.
	Copied!

Copy slurm-9249639_415.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_415.out.
	Copied!

Copy slurm-9249639_416.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_416.out.
	Copied!

Copy slurm-9249639_417.out 
	 to /ho

	Copied!

Copy slurm-9249639_485.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_485.out.
	Copied!

Copy slurm-9249639_486.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_486.out.
	Copied!

Copy slurm-9249639_487.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_487.out.
	Copied!

Copy slurm-9249639_488.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_488.out.
	Copied!

Copy slurm-9249639_489.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_489.out.
	Copied!

Copy slurm-9249639_490.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_490.out.
	Copied!

Copy slurm-9249639_491.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_491.out.
	Copied!

Copy slurm-9249639_492.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_492.out.
	Copied!

Copy slurm-9249639_493.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_493.out.
	Copied!

Copy slurm-9249639_494.out 

	Copied!

Copy slurm-9249639_563.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_563.out.
	Copied!

Copy slurm-9249639_564.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_564.out.
	Copied!

Copy slurm-9249639_565.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_565.out.
	Copied!

Copy slurm-9249639_566.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_566.out.
	Copied!

Copy slurm-9249639_567.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_567.out.
	Copied!

Copy slurm-9249639_568.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_568.out.
	Copied!

Copy slurm-9249639_569.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_569.out.
	Copied!

Copy slurm-9249639_570.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_570.out.
	Copied!

Copy slurm-9249639_571.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_571.out.
	Copied!

Copy slurm-9249639_572.out 

	Copied!

Copy slurm-9249639_641.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_641.out.
	Copied!

Copy slurm-9249639_642.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_642.out.
	Copied!

Copy slurm-9249639_643.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_643.out.
	Copied!

Copy slurm-9249639_644.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_644.out.
	Copied!

Copy slurm-9249639_645.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_645.out.
	Copied!

Copy slurm-9249639_646.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_646.out.
	Copied!

Copy slurm-9249639_647.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_647.out.
	Copied!

Copy slurm-9249639_648.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_648.out.
	Copied!

Copy slurm-9249639_649.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_649.out.
	Copied!

Copy slurm-9249639_650.out 

	Copied!

Copy slurm-9249639_718.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_718.out.
	Copied!

Copy slurm-9249639_719.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_719.out.
	Copied!

Copy slurm-9249639_720.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_720.out.
	Copied!

Copy slurm-9249639_721.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_721.out.
	Copied!

Copy slurm-9249639_722.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_722.out.
	Copied!

Copy slurm-9249639_723.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_723.out.
	Copied!

Copy slurm-9249639_724.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_724.out.
	Copied!

Copy slurm-9249639_725.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_725.out.
	Copied!

Copy slurm-9249639_726.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_726.out.
	Copied!

Copy slurm-9249639_727.out 

	Copied!

Copy slurm-9249639_795.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_795.out.
	Copied!

Copy slurm-9249639_796.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_796.out.
	Copied!

Copy slurm-9249639_797.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_797.out.
	Copied!

Copy slurm-9249639_798.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_798.out.
	Copied!

Copy slurm-9249639_799.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_799.out.
	Copied!

Copy slurm-9249639_800.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_800.out.
	Copied!

Copy slurm-9249639_801.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_801.out.
	Copied!

Copy slurm-9249639_802.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_802.out.
	Copied!

Copy slurm-9249639_803.out 
	 to /home/brian/Dropbox/PSC/Error_logs 
	 as slurm-9249639_803.out.
	Copied!

Copy slurm-9249639_804.out 

In [47]:
#Check each file in Error_logs for errors:
import os
error_dir = '/home/brian/Dropbox/PSC/Error_logs'
def check_if_string_in_file(file_name, string_to_search):
    """ Check if any line in the file contains given string """
    # Open the file in read only mode
    with open(file_name, 'r') as read_obj:
        # Read all lines in the file one by one
        for line in read_obj:
            # For each line, check if line contains the string
            if string_to_search in line:
                return True, line
    return False, None

err_files=[]
files = os.listdir(error_dir)
err_line=[]
array_cmd='--array='
for file in files:
    if ('.out' in file) and ('.swp' not in file):
        has_error,line=check_if_string_in_file(error_dir + '/' + file,'error')
        if has_error:
            err_files.append(file)
            err_line.append(line)
            #print('Error found in %s:' % file)
            #print('\t%s' % line[:-1])
            if 'DUE TO TIME LIMIT' in line:
                file_num=file.split('_')[1].split('.')[0]
                print('\tTIME OUT ERR DETECT in %s.\n' % file_num)
                array_cmd += '%s,' % file_num
            else:
                print('\tUNIDENTIFIED ERROR.\n')
    else:
        print('%s excluded from analysis' % str(file))
print(array_cmd[:-1])
srvr.close()

	TIME OUT ERR DETECT in 700.

	TIME OUT ERR DETECT in 679.

	TIME OUT ERR DETECT in 626.

	TIME OUT ERR DETECT in 708.

	TIME OUT ERR DETECT in 604.

	TIME OUT ERR DETECT in 686.

	TIME OUT ERR DETECT in 682.

	TIME OUT ERR DETECT in 622.

	TIME OUT ERR DETECT in 610.

	TIME OUT ERR DETECT in 615.

	TIME OUT ERR DETECT in 609.

	TIME OUT ERR DETECT in 605.

	TIME OUT ERR DETECT in 634.

	TIME OUT ERR DETECT in 633.

	TIME OUT ERR DETECT in 602.

	UNIDENTIFIED ERROR.

	TIME OUT ERR DETECT in 624.

.slurm-9127108_5.out.swp excluded from analysis
	TIME OUT ERR DETECT in 614.

	TIME OUT ERR DETECT in 618.

	TIME OUT ERR DETECT in 621.

	TIME OUT ERR DETECT in 608.

--array=700,679,626,708,604,686,682,622,610,615,609,605,634,633,602,624,614,618,621,608


In [37]:
srvr= pysftp.Connection('data.bridges.psc.edu',username='bisett',password='deepL4bCutta',port=22)
remote_vid='/pylon5/ib5phrp/bisett/projects/psc_analyze-BRI-2020-05-01/videos'

#First generate list of videos:
files=srvr.listdir(remote_vid)
vids=[v for v in files if '.mpg' in v]
for vid in vids:
    print(vid)
    
srvr.close()

GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_10x30_AG4361-1-JS041119.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_10x30_AG4361-3-JS042219.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_10x30_AG4361-4-JS042219.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_10x30_AG4446-2-JS040319.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_trig-r_AG4361-1-JS050619.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_trig-r_AG4361-3-JS050619.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_trig-r_AG4361-4-JS050619.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_trig-r_AG4361-5-JS050819.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_trig-r_AG4446-2-JS040919.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_zone-1_AG4361-1-JS041219.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_zone-1_AG4361-3-JS042319.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_zone-1_AG4361-4-JS042319.mpg
GPe-and-SNr_Naive_PV-PV_Arch-caspase_Bilateral_zone-1_AG4361-5-JS050119.mpg
GPe-and-SNr_Naiv